# Librairies


In [1]:
import cv2
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from base64 import b64decode
from IPython.display import display, Javascript
from google.colab.output import eval_js
import pickle


# Création du dataset

In [2]:
# on crée le répertoire data s'il n'existe pas
if not os.path.exists("data"):
    os.mkdir("data")


def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button'); // bouton pour capture d'image
        capture.textContent = 'Capture';
        div.appendChild(capture);

        // permet l'affichage du flux vidéo de notre caméra en temps réel
        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // l'exécution du script s'interromp jusqu'à ce qu'un clic pour la capture soit effectué
        await new Promise((resolve) => capture.onclick = resolve);

        //  capture de l'image, arrêt de la caméra, mise en format jpeg de l'image
        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    # On exécute le script JavaScript et on réceptionne l'image capturée sous forme de chaîne de données base64
    data = eval_js('takePhoto({})'.format(quality))
    # on décode la chaîne base64 pour obtenir les données de l'image en binaire
    binary = b64decode(data.split(',')[1])
    # Sauvegarde des données binaires dans un fichier image
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Chargement d'un modèle pré-entraîné de détection de visages fourni par OpenCV
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Nombre d'images qu'on collecte pour la reconnaissance faciale
nb_images_collectees = 10

# Initialisation des listes pour stocker les données des visage
donnees_visage = []
noms = []# Initialisation d'un liste vide pour stocker les noms correspondant aux visages capturés
# Demande à l'utilisateur de saisir le nom de la personne qui sera prise en photo, ce même nom sra associéà toutes les images capturées
nom = input("Entrez votre nom: ")

# Boucle pour capturer et traiter les images selon le nombre d'image que l'on a défini plus haut
for i in range(nb_images_collectees):
    filename = take_photo()  # Capture d'une nouvelle photo
    image = cv2.imread(filename)# lecture de l'img capturée
    gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)#image mise en niveaux de gris
    #Utilisation du modèle de détection de visages pour trouver les visages dans l'image
    coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

    # Traitement de chaque visage détecté dans l'image
    for (x, y, w, h) in coordonnees_visage:
      # Pour chaque visage détecté, extrait, redimensionne, et aplati le visage avant de l'ajouter à la liste des données de visage
        visage = image[y:y+h, x:x+w]
        visage_redimensionne = cv2.resize(visage, (64, 64))
         # Aplatissement de l'image redimensionnée pour la préparation avant l'entraînement du modèle.
        donnees_visage.append(visage_redimensionne.flatten())
        noms.append(nom)  # Association du nom de l'utilisateur au visage capturé
        break


Entrez votre nom: beyonce


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Algorithme et exécution

In [3]:
# Conversion des listes en tableaux numpy
donnees_visage = np.array(donnees_visage)
noms = np.array(noms)

In [8]:
# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(donnees_visage, noms, test_size=0.2, random_state=42)

#on génère des données aléatoires pour simuler une fausse classe : sinon on avait une erreur : le modèle devait s'entraîner avec des données d'une autre classe également
autre_classe = np.random.rand(10, 64*64*3)  # Générer des données aléatoires
donnees_visage = np.concatenate((donnees_visage, autre_classe), axis=0)
noms = np.concatenate((noms, np.array(["Non-JayZ"]*10)))

# Mélanger les données pour que l'ordre soit aléatoire
indices = np.arange(donnees_visage.shape[0])
np.random.shuffle(indices)
donnees_visage = donnees_visage[indices]
noms = noms[indices]

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(donnees_visage, noms, test_size=0.2, random_state=42)

# Entraînement du modèle
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

#fin fausse classe
# Entraînement du modèle de régression logistique
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
# Fonction pour prédire le nom à partir d'une nouvelle capture de visage
def predict_face():
    filename = take_photo()  # Capture d'une nouvelle photo

    image = cv2.imread(filename)# lecture de l'img capturée
    gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)#image mise en niveaux de gris
     # Utilisation du modèle de détection de visages pour trouver les visages dans l'image convertie en niveaux de gris.
    coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

    for (x, y, w, h) in coordonnees_visage:
        # Extraction du visage détecté à partir de l'image originale
        visage = image[y:y+h, x:x+w]

        # Redimensionnement du visage pour que ç corresponde à la taille qu'il faut pour le modèle de reconnaissance faciale + applatissement du bisage
        visage_redimensionne = cv2.resize(visage, (64, 64)).flatten().reshape(1, -1)
        # Utilisation du modèle de régression logistique entraîné (`lr`) pour prédire le nom de la personne à qui ce visage appartient
        # La prédiction renvoie le nom associé au visage plus correspondant
        prediction = lr.predict(visage_redimensionne)
        # Affichage du nom prédit.
        print(f"Visage reconnu comme : {prediction[0]}")
        break
    else:  # Si aucun visage n'est détecté
        print("Aucun visage détecté.")

predict_face()

<IPython.core.display.Javascript object>

Visage reconnu comme : khaoula
